# Exploratory Data Analysis of ~20k tweets from 9/11/2016 to 12/12/2016.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import indicoio as indico
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
df = pd.read_csv('tweets.csv')

In [5]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True, errors = 'ignore')

In [6]:
df.columns

Index(['user id', 'name', 'handle', 'date', 'retweets', 'favorites', 'text',
       'id', 'permalink', 'language', 'followers_count', 'location',
       'sentiment'],
      dtype='object')

In [13]:
df.head()

,user id,name,handle,date,retweets,favorites,text,id,permalink,language,followers_count,location,sentiment
0,7.108627e+17,The gamer,VenkateshaPanc2,2016-11-09 23:38:24,0.0,0.0,If common man is supporting the decision why c...,7.964142e+17,https://twitter.com/VenkateshaPanc2/status/796...,en,6.0,"Indore, India",0.712256
1,7.291956e+07,Uma Kant Singh,umakantsingh_in,2016-11-09 23:38:42,0.0,0.0,"#DeMonetisation,Not govt of common man,PM who ...",7.964142e+17,https://twitter.com/umakantsingh_in/status/796...,en,455.0,New Delhi,0.433632
2,2.381901e+08,Antony bothagar,Antonybothagar,2016-11-09 23:38:46,0.0,1.0,I am already suffering to get changes for ₹ 10...,7.964142e+17,https://twitter.com/Antonybothagar/status/7964...,en,76.0,NaN,0.844959
3,2.433278e+09,Abhishek Sharma,skyneeldotcom,2016-11-09 23:38:53,1.0,1.0,#APPSC #Recruitment 2016 – Apply Online for 98...,7.964143e+17,https://twitter.com/skyneeldotcom/status/79641...,en,6288.0,NaN,0.568757
4,3.246635e+09,abdul rahman,RahmanAbdul2603,2016-11-09 23:39:21,0.0,0.0,#DeMonetisation almst everybody knows fr a mar...,7.964144e+17,https://twitter.com/RahmanAbdul2603/status/796...,en,48.0,"Saharanpur, India",0.080881


## Q1. What was the most retweeted tweet in the last 1 month?

In [10]:
df.loc[df['retweets'].argmax()]['text']

'A total of 11 bank officers died of stress since Govt rolled out #demonetisation, says union representing 2.5L bank officers'

## A. 'A total of 11 bank officers died of stress since Govt rolled out #demonetisation, says union representing 2.5L bank officers', tweeted by the Indian Express

## Q2. The most favourited tweet?

In [15]:
df.loc[df['favorites'].argmax()]

user id                                                  3.95612e+09
name                                                  Mitesh Patel 🎗
handle                                                  _MiteshPatel
date                                             2016-11-11 19:03:24
retweets                                                        1495
favorites                                                       1303
text               Ahmedabad's @modibhatiakisor dares @AmitShah t...
id                                                        7.9707e+17
permalink          https://twitter.com/_MiteshPatel/status/797069...
language                                                          en
followers_count                                                 4731
location                                  Ahmedabad, Gujarat, India.
sentiment                                                   0.523209
Name: 1468, dtype: object